In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#%matplotlib inline
sns.set()
from sklearn.feature_selection import mutual_info_regression
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/data-storm-4/Data-dictionary.xlsx
/kaggle/input/data-storm-4/DataStorm4.0_KaggleProblemStatement.pdf
/kaggle/input/data-storm-4/Store-info.csv
/kaggle/input/data-storm-4/Testing-data.csv
/kaggle/input/data-storm-4/Historical-transaction-data.csv


In [2]:
df = pd.read_csv('/kaggle/input/data-storm-4/Historical-transaction-data.csv')
store_info_data = pd.read_csv('/kaggle/input/data-storm-4/Store-info.csv')
test_data = pd.read_csv('/kaggle/input/data-storm-4/Testing-data.csv')

In [3]:
test_data.count()

shop_id         24
shop_profile     0
dtype: int64

In [4]:
store_info_data.isna().sum()

shop_id             0
shop_area_sq_ft     0
shop_profile       24
dtype: int64

In [5]:
# # Create a mapping between item_price values and item_description values
# price_to_description = df.groupby('item_description')['item_price'].apply(lambda x: x.mode()[0]).to_dict()

# # Fill missing values in 'item_description' column using mapping
# df['item_description'].fillna(df['item_price'].map(price_to_description), inplace=True)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 473974 entries, 0 to 473973
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   item_description  438046 non-null  object 
 1   transaction_date  473974 non-null  object 
 2   invoice_id        467654 non-null  float64
 3   customer_id       473974 non-null  object 
 4   shop_id           473974 non-null  object 
 5   item_price        473974 non-null  int64  
 6   quantity_sold     473974 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 25.3+ MB


In [7]:
print(df.isnull().sum())
df.fillna(method='ffill',inplace=True)
print(df.isnull().sum())

item_description    35928
transaction_date        0
invoice_id           6320
customer_id             0
shop_id                 0
item_price              0
quantity_sold           0
dtype: int64
item_description    0
transaction_date    0
invoice_id          0
customer_id         0
shop_id             0
item_price          0
quantity_sold       0
dtype: int64


In [8]:
df['quantity_sold'] = df.quantity_sold.abs()

In [9]:
df[df.duplicated()].shape[0]
df.drop_duplicates(inplace=True)
print(df[df.duplicated()].shape[0])

0


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 442747 entries, 0 to 473973
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   item_description  442747 non-null  object 
 1   transaction_date  442747 non-null  object 
 2   invoice_id        442747 non-null  float64
 3   customer_id       442747 non-null  object 
 4   shop_id           442747 non-null  object 
 5   item_price        442747 non-null  int64  
 6   quantity_sold     442747 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 27.0+ MB


In [11]:
# Group transactions by item description
grouped = df.groupby('item_description')

# Add a new column for item price with the mode value of each group
df['item_price'] = grouped['item_price'].transform(lambda x: x.mode()[0])

In [12]:
print(grouped.head())

            item_description          transaction_date  invoice_id  \
0         ORANGE BARLEY 1.5L  2021-12-11T00:00:00.000Z       147.0   
1           GINGER BEER 1.5L  2021-10-17T00:00:00.000Z       371.0   
2            TONIC PET 500ML  2021-12-13T00:00:00.000Z       484.0   
3              CREAM SODA 1L  2021-12-13T00:00:00.000Z       484.0   
4      STRAWBERRY MILK 180ML  2021-10-23T00:00:00.000Z      1310.0   
...                      ...                       ...         ...   
65149   CHOCOLATE MILK 180ML  2021-12-14T00:00:00.000Z   2007816.0   
65151   CHOCOLATE MILK 180ML  2021-10-22T00:00:00.000Z   2016254.0   
75311    LIME CRUSH JUICE 1L  2021-12-04T00:00:00.000Z   1310839.0   
75313    LIME CRUSH JUICE 1L  2021-12-04T00:00:00.000Z   1310839.0   
75314    LIME CRUSH JUICE 1L  2021-12-04T00:00:00.000Z   1310839.0   

      customer_id  shop_id  item_price  quantity_sold  
0            BGXA  SHOP008         220              2  
1            IA25  SHOP112         220         

In [13]:
df.item_price.unique()

array([220, 160, 150,  70, 270,  60, 290, 120, 100,  35, 110,  80, 140,
       650])

In [14]:
# df_test = dataframe[dataframe['shop_profile'].isnull()]

In [15]:
# df_train = dataframe[dataframe['shop_profile'].notnull()]

In [16]:
# df = df_train

In [17]:
# df =  df.reset_index(drop=True)

In [18]:
from datetime import datetime

def extract_date(date_str):
    date_obj = datetime.strptime(date_str, "%Y-%m-%dT%H:%M:%S.%fZ")
    return date_obj.strftime("%Y-%m-%d")

df["transaction_date"] = df["transaction_date"].apply(extract_date)

In [19]:
# Convert the date column to a Pandas datetime object
df["transaction_date"] = pd.to_datetime(df["transaction_date"])

# Extract the year, month, day, and week of the year into separate columns
df["month"] = df["transaction_date"].dt.month
df["day"] = df["transaction_date"].dt.day
df["week"] = df["transaction_date"].dt.isocalendar().week

In [20]:
# for colname in df.select_dtypes(["category", "object"]):
#         df[colname], _ = df[colname].factorize()

In [21]:
# Group the data by shop and customer, and count the number of transactions
transactions = df.groupby(["shop_id", "customer_id"])["invoice_id"].nunique()

# Count the number of customers who made more than one transaction at each shop
repeat_customers = transactions[transactions > 1].groupby("shop_id").count()

# Count the total number of unique customers who made a transaction at each shop
total_customers = transactions.groupby("shop_id").count()

# Calculate the customer loyalty rate for each shop
loyalty_rate = repeat_customers / total_customers

# Reset the index of the loyalty_rate Series and rename the columns
loyalty_rate = loyalty_rate.reset_index()
loyalty_rate.columns = ["shop_id", "loyalty_rate"]

# Merge the loyalty rate with the original DataFrame on the "shop_id" column
df = pd.merge(df, loyalty_rate, on="shop_id")

In [22]:
df = df.drop(["transaction_date","invoice_id"], axis= 1)

In [23]:
df.week.unique()

<IntegerArray>
[49, 50, 41, 42, 43, 44, 45, 46, 47, 48]
Length: 10, dtype: UInt32

In [24]:
df.shop_id.unique()

array(['SHOP008', 'SHOP112', 'SHOP055', 'SHOP003', 'SHOP071', 'SHOP123',
       'SHOP087', 'SHOP090', 'SHOP022', 'SHOP047', 'SHOP012', 'SHOP054',
       'SHOP040', 'SHOP023', 'SHOP115', 'SHOP104', 'SHOP097', 'SHOP103',
       'SHOP125', 'SHOP034', 'SHOP061', 'SHOP001', 'SHOP080', 'SHOP116',
       'SHOP075', 'SHOP093', 'SHOP096', 'SHOP026', 'SHOP032', 'SHOP107',
       'SHOP044', 'SHOP017', 'SHOP037', 'SHOP099', 'SHOP051', 'SHOP122',
       'SHOP088', 'SHOP121', 'SHOP046', 'SHOP067', 'SHOP098', 'SHOP065',
       'SHOP094', 'SHOP095', 'SHOP091', 'SHOP041', 'SHOP049', 'SHOP042',
       'SHOP004', 'SHOP076', 'SHOP073', 'SHOP027', 'SHOP083', 'SHOP043',
       'SHOP089', 'SHOP025', 'SHOP024', 'SHOP005', 'SHOP078', 'SHOP063',
       'SHOP021', 'SHOP086', 'SHOP105', 'SHOP074', 'SHOP062', 'SHOP079',
       'SHOP068', 'SHOP011', 'SHOP066', 'SHOP028', 'SHOP108', 'SHOP085',
       'SHOP038', 'SHOP060', 'SHOP111', 'SHOP059', 'SHOP118', 'SHOP013',
       'SHOP014', 'SHOP052', 'SHOP007', 'SHOP119', 

In [25]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

df['total_sales'] = df['item_price'] * df['quantity_sold']
#df['last_month_sale'] = df['']
# scaler = StandardScaler()
# scaled_data = scaler.fit_transform(df[['shop_area_sq_ft', 'item_price', 'quantity_sold', 'total_sales']])

# df = df.drop(['shop_area_sq_ft', 'item_price', 'quantity_sold', 'total_sales'], axis=1)

# scaled_df = pd.DataFrame(scaled_data, columns=['shop_area_sq_ft', 'item_price', 'quantity_sold', 'total_sales'])

# df = pd.concat([df, scaled_df], axis=1)

# print(df.head())

In [26]:
df.head()

,item_description,customer_id,shop_id,item_price,quantity_sold,month,day,week,loyalty_rate,total_sales
0,ORANGE BARLEY 1.5L,BGXA,SHOP008,220,2,12,11,49,0.058201,440
1,TONIC PET 500ML,VN7V,SHOP008,160,2,12,13,50,0.058201,320
2,CREAM SODA 1L,VN7V,SHOP008,150,2,12,13,50,0.058201,300
3,GINGER BEER 1.5L,VT9C,SHOP008,220,1,12,10,49,0.058201,220
4,GINGER BEER 1.5L,8QLS,SHOP008,220,1,12,10,49,0.058201,220


In [27]:
# Filter for week 50
# total_sales_week_50 = df[(df['week'] == 49) | (df['week'] == 50)].groupby('shop_id')['total_sales'].sum()
total_sales_week_50 = df.loc[df.week.isin ([48,49,50])].groupby('shop_id')['total_sales'].sum()

# Display the result
# print(total_sales_week_50)

In [28]:
# Group transactions by shop_id
shop_transactions = df.groupby('shop_id')

# Calculate summary statistics for each shop
shop_summary = shop_transactions.agg({
    'item_description': 'nunique', # number of unique items sold
    'quantity_sold': 'sum', # total quantity sold
    'total_sales': 'sum', # total sales
})

# Calculate loyalty rate mean for each shop
loyalty_rate_mean = df.groupby('shop_id')['loyalty_rate'].mean()

# Merge loyalty rate mean into shop_summary
shop_summary = shop_summary.merge(loyalty_rate_mean, on='shop_id')

# Group the data by shop and calculate the average item price
avg_item_price = df.groupby("shop_id")["item_price"].mean()

# Reset the index of the avg_item_price Series and rename the columns
avg_item_price = avg_item_price.reset_index()
avg_item_price.columns = ["shop_id", "avg_item_price"]

# Merge the avg_item_price with the original DataFrame on the "shop_id" column
shop_summary = shop_summary.merge(avg_item_price, on='shop_id')



In [29]:
shop_summary = shop_summary.merge(total_sales_week_50, on='shop_id')
shop_summary = shop_summary.rename(columns={'total_sales_x': 'total_sales', 'total_sales_y': 'total_sales_week_50'})

In [30]:
# Flatten column names
shop_summary.columns = [''.join(col).strip() for col in shop_summary.columns.values]

# Reset index to make shop_id a column again
shop_summary = shop_summary.reset_index()

In [31]:
shop_summary

,index,shop_id,item_description,quantity_sold,total_sales,loyalty_rate,avg_item_price,total_sales_week_50
0,0,SHOP001,35,3752,615770,0.231664,173.788038,247650
1,1,SHOP002,35,9042,1532655,0.234381,172.369125,611790
2,2,SHOP003,36,6548,1063720,0.218023,164.849547,366665
3,3,SHOP004,35,5969,971270,0.270069,165.656991,380155
4,4,SHOP005,36,7301,1245750,0.261916,172.435865,385900
...,...,...,...,...,...,...,...,...
119,119,SHOP123,36,3631,590900,0.238136,169.319710,202225
120,120,SHOP124,35,8506,1309570,0.248575,164.398193,448430
121,121,SHOP125,36,4101,530815,0.212766,138.121336,198555
122,122,SHOP126,33,6246,919030,0.219333,156.981070,296315


In [32]:
shop_summary = pd.merge(shop_summary, store_info_data, on='shop_id')

In [33]:
shop_summary

,index,shop_id,item_description,quantity_sold,total_sales,loyalty_rate,avg_item_price,total_sales_week_50,shop_area_sq_ft,shop_profile
0,0,SHOP001,35,3752,615770,0.231664,173.788038,247650,580,Low
1,1,SHOP002,35,9042,1532655,0.234381,172.369125,611790,637,NaN
2,2,SHOP003,36,6548,1063720,0.218023,164.849547,366665,810,High
3,3,SHOP004,35,5969,971270,0.270069,165.656991,380155,516,Low
4,4,SHOP005,36,7301,1245750,0.261916,172.435865,385900,617,Low
...,...,...,...,...,...,...,...,...,...,...
119,119,SHOP123,36,3631,590900,0.238136,169.319710,202225,735,Moderate
120,120,SHOP124,35,8506,1309570,0.248575,164.398193,448430,606,High
121,121,SHOP125,36,4101,530815,0.212766,138.121336,198555,1028,High
122,122,SHOP126,33,6246,919030,0.219333,156.981070,296315,328,Moderate


In [34]:
shop_summary

,index,shop_id,item_description,quantity_sold,total_sales,loyalty_rate,avg_item_price,total_sales_week_50,shop_area_sq_ft,shop_profile
0,0,SHOP001,35,3752,615770,0.231664,173.788038,247650,580,Low
1,1,SHOP002,35,9042,1532655,0.234381,172.369125,611790,637,NaN
2,2,SHOP003,36,6548,1063720,0.218023,164.849547,366665,810,High
3,3,SHOP004,35,5969,971270,0.270069,165.656991,380155,516,Low
4,4,SHOP005,36,7301,1245750,0.261916,172.435865,385900,617,Low
...,...,...,...,...,...,...,...,...,...,...
119,119,SHOP123,36,3631,590900,0.238136,169.319710,202225,735,Moderate
120,120,SHOP124,35,8506,1309570,0.248575,164.398193,448430,606,High
121,121,SHOP125,36,4101,530815,0.212766,138.121336,198555,1028,High
122,122,SHOP126,33,6246,919030,0.219333,156.981070,296315,328,Moderate


In [35]:
shop_summary['sales_per_feet'] = shop_summary['total_sales'] / shop_summary['shop_area_sq_ft']
shop_summary['sales_per_week'] = shop_summary['total_sales_week_50'] / 3
shop_summary['sales_per_week_total'] = shop_summary['total_sales'] / 10

# shop_summary.to_csv('shop.csv', index=False)

In [36]:
shop_summary

,index,shop_id,item_description,quantity_sold,total_sales,loyalty_rate,avg_item_price,total_sales_week_50,shop_area_sq_ft,shop_profile,sales_per_feet,sales_per_week,sales_per_week_total
0,0,SHOP001,35,3752,615770,0.231664,173.788038,247650,580,Low,1061.672414,82550.000000,61577.0
1,1,SHOP002,35,9042,1532655,0.234381,172.369125,611790,637,NaN,2406.051805,203930.000000,153265.5
2,2,SHOP003,36,6548,1063720,0.218023,164.849547,366665,810,High,1313.234568,122221.666667,106372.0
3,3,SHOP004,35,5969,971270,0.270069,165.656991,380155,516,Low,1882.306202,126718.333333,97127.0
4,4,SHOP005,36,7301,1245750,0.261916,172.435865,385900,617,Low,2019.043760,128633.333333,124575.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,119,SHOP123,36,3631,590900,0.238136,169.319710,202225,735,Moderate,803.945578,67408.333333,59090.0
120,120,SHOP124,35,8506,1309570,0.248575,164.398193,448430,606,High,2161.006601,149476.666667,130957.0
121,121,SHOP125,36,4101,530815,0.212766,138.121336,198555,1028,High,516.357004,66185.000000,53081.5
122,122,SHOP126,33,6246,919030,0.219333,156.981070,296315,328,Moderate,2801.920732,98771.666667,91903.0


In [37]:
shop_summary.item_description.unique()

array([35, 36, 31, 32, 33, 34])

In [38]:
scaler = StandardScaler()
scaled_data = scaler.fit_transform(shop_summary[['quantity_sold', 'total_sales','shop_area_sq_ft', "avg_item_price",'sales_per_feet','loyalty_rate','sales_per_week','sales_per_week_total']])

shop_summary = shop_summary.drop(['quantity_sold', 'total_sales','shop_area_sq_ft','sales_per_feet', "avg_item_price",'loyalty_rate','sales_per_week','sales_per_week_total'], axis=1)

scaled_df = pd.DataFrame(scaled_data, columns=['quantity_sold', 'total_sales', 'shop_area_sq_ft', "avg_item_price",'sales_per_feet','loyalty_rate','sales_per_week','sales_per_week_total'])

shop_summary = pd.concat([shop_summary, scaled_df], axis=1)

In [39]:
df_test = shop_summary[shop_summary['shop_profile'].isnull()]
df_test =  df_test.reset_index(drop=True)

In [40]:
df_train = shop_summary[shop_summary['shop_profile'].notnull()]
df_train =  df_train.reset_index(drop=True)

In [41]:
df_x = df_train.drop('shop_profile', axis =1)
df_y = df_train['shop_profile']

In [42]:
df_x = df_x[['shop_id', 'sales_per_feet', "avg_item_price",'loyalty_rate','sales_per_week','sales_per_week_total']]

In [43]:
df1_x = df_test.drop('shop_profile', axis =1)
df1_x = df1_x[['shop_id','sales_per_feet', "avg_item_price",'loyalty_rate','sales_per_week','sales_per_week_total']]

In [44]:
X_train = df_x.drop('shop_id', axis =1 )
X_test = df1_x.drop('shop_id', axis =1 )
# X_train = df_x
# X_test = df1_x

In [45]:
# np.random.seed(123) 

# X_train, X_test, y_train, y_test = train_test_split(df_x, df_y,
#                                                    test_size= 0.2,
#                                                    shuffle= True, #shuffle the data to avoid bias
#                                                    random_state= 5)

# print(f'training set size: {X_train.shape[0]} samples \ntest set size: {X_test.shape[0]} samples')

y_train = df_y

In [46]:
X_test

,sales_per_feet,avg_item_price,loyalty_rate,sales_per_week,sales_per_week_total
0,0.851941,0.712769,-0.140741,1.918410,1.295365
1,1.684692,1.323037,1.369126,0.486918,0.292583
2,0.301220,0.867482,-0.369936,0.396334,0.482730
3,-1.054332,-0.199199,0.161172,-1.208169,-0.936680
4,0.011691,0.513987,-0.501396,0.352629,0.018189
5,0.561888,-0.107126,1.972401,0.065616,0.202938
6,0.497379,0.308617,0.412835,-0.547616,-0.661931
7,-1.303611,-0.913561,-1.190945,-1.229249,-1.142157
8,-0.024560,-0.643993,1.009538,-0.157802,-0.319757
9,2.056268,1.512728,3.054998,1.153425,0.591304


In [47]:
# max_depths = [2*i for i in range(1,15)]
# min_samples_splits = [2,4,8]
# min_samples_leafs =  [1,2,4]
# n_estimator = 100
# best_score = 0

# for max_depth in max_depths:
#     for min_samples_split in min_samples_splits:
#         for min_samples_leaf in min_samples_leafs:
#             random_forest = RandomForestClassifier( max_depth=max_depth, min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf)

#             accuracy_list = cross_val_score(random_forest, X_train, y_train, cv=10)
#             accuracy = accuracy_list.mean()

#             if accuracy > best_score:
#                 best_score = accuracy
#                 best_params = {'max_depth': max_depth, 'min_samples_split': min_samples_split, 'min_samples_leaf': min_samples_leaf}
#                 print(f'Best hyperparameters: {best_params}')
#                 print(f'Accuracy: {best_score}')


# print(f'Best hyperparameters: {best_params}')
# print(f'Accuracy: {best_score}')

In [48]:
randomForest = RandomForestClassifier(max_depth= 12, min_samples_split= 4, min_samples_leaf= 1)
# max_depth= 6, min_samples_split= 4, min_samples_leaf= 1
# n_estimators = 100, max_depth=16, min_samples_split=2, min_samples_leaf=1

randomForest.fit(X_train, y_train)
y_pred = randomForest.predict(X_test)

In [49]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(randomForest, X_train, y_train, cv=10)                                   
print("Cross-validation scores:", scores)
print("Mean cross-validation score:", scores.mean())

Cross-validation scores: [0.7 0.6 0.7 0.4 0.5 0.5 0.8 0.6 0.7 0.7]
Mean cross-validation score: 0.62


In [50]:
# import xgboost as xgb
# from sklearn.datasets import load_breast_cancer
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# model = xgb.XGBClassifier()

# model.fit(X_train, y_train)

# y_pred = model.predict(X_test)

In [51]:
shop_profile = pd.DataFrame({'shop_profile': y_pred})

df_concat = pd.concat([df1_x, shop_profile], axis=1)


In [52]:
output = df_concat[['shop_id','shop_profile']]
print(output)

    shop_id shop_profile
0   SHOP002         High
1   SHOP019          Low
2   SHOP023         High
3   SHOP024          Low
4   SHOP029          Low
5   SHOP030          Low
6   SHOP038          Low
7   SHOP044     Moderate
8   SHOP046     Moderate
9   SHOP050     Moderate
10  SHOP056          Low
11  SHOP061          Low
12  SHOP070     Moderate
13  SHOP074     Moderate
14  SHOP076          Low
15  SHOP080          Low
16  SHOP081         High
17  SHOP087     Moderate
18  SHOP092          Low
19  SHOP096          Low
20  SHOP097     Moderate
21  SHOP107         High
22  SHOP108          Low
23  SHOP114     Moderate


In [53]:
# Save test predictions to file

output.to_csv('submission.csv', index=False)

In [54]:
# accuracy = accuracy_score(y_test, y_pred)
# precision = precision_score(y_test, y_pred, average='weighted')
# recall = recall_score(y_test, y_pred, average='weighted')
# f1score = f1_score(y_test, y_pred, average='weighted')

# print("Accuracy:", accuracy)
# print("Precision:", precision)
# print("Recall:", recall)
# print("F1-score:", f1score)

In [55]:
# from sklearn.datasets import load_iris
# from sklearn.model_selection import cross_val_score
# from sklearn.neighbors import KNeighborsClassifier

# knn = KNeighborsClassifier(n_neighbors=5)

# # Fit the model to the training data
# knn.fit(X_train, y_train)

# # Use the fitted model to make predictions on the test data
# y_pred = knn.predict(X_test)

In [56]:
# accuracy = accuracy_score(y_test, y_pred)
# precision = precision_score(y_test, y_pred, average='weighted')
# recall = recall_score(y_test, y_pred, average='weighted')
# f1score = f1_score(y_test, y_pred, average='weighted')

# print("Accuracy:", accuracy)
# print("Precision:", precision)
# print("Recall:", recall)
# print("F1-score:", f1score)